In [12]:
pip install tensorflow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

train = pd.read_csv('train_df.csv')
test = pd.read_csv('test_df.csv')

# cleaning and sets
validation_subjects = {"co2a0000377", "co2a0000364", "co2c0000342", "co2c0000345"}
val = test[test['subject'].isin(validation_subjects)].drop(columns=['sample', 'match', 'err', 'object'])
test = test[~test['subject'].isin(validation_subjects)]

train = train.drop(columns=['sample', 'match', 'err', 'object'])
test = test.drop(columns=['sample', 'match', 'err', 'object'])

X_train = train.drop(columns=['alcoholic', 'subject'])
y_train = train[['alcoholic']]

X_test = test.drop(columns=['alcoholic', 'subject'])
y_test = test[['alcoholic']]

X_val = val.drop(columns=['alcoholic', 'subject'])
y_val = val[['alcoholic']]

# normalize and reshape
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# model layers and compile
model = models.Sequential()

model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(layers.MaxPooling1D(2))

model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/Users/dpark_23/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

Epoch 1/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.7986 - loss: 0.4121 - val_accuracy: 0.8939 - val_loss: 0.2519
Epoch 2/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9210 - loss: 0.1934 - val_accuracy: 0.9044 - val_loss: 0.2296
Epoch 3/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9468 - loss: 0.1320 - val_accuracy: 0.9019 - val_loss: 0.2558
Epoch 4/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9619 - loss: 0.0994 - val_accuracy: 0.9062 - val_loss: 0.2521
Epoch 5/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9709 - loss: 0.0769 - val_accuracy: 0.9187 - val_loss: 0.2239
Epoch 6/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9763 - loss: 0.0621 - val_accuracy: 0.9192 - val_loss: 0.2402
Epoch 7/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9804 - loss: 0.0528 - val_accuracy: 0.9229 - val_loss: 0.2248
Epoch 8/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9834 - loss: 0

In [21]:
test_acc

0.8722981810569763